## Preparations

In [109]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import numpy as np
from scipy.spatial import distance
from platform import python_version
#you should install sentence_transformers
#pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(device)
print(python_version())

cuda
3.10.8


## Load & Preprocess Data 

Let's have a look to our dialydialog dataset : https://www.aclweb.org/anthology/I17-1099/

In [110]:
corpus_name = "dailydialog"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding="utf-8") as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "dialogues_text.txt"))

The kitchen stinks . __eou__ I'll throw out the garbage . __eou__

So Dick , how about getting some coffee for tonight ? __eou__ Coffee ? I don ’ t honestly like that kind of stuff . __eou__ Come on , you can at least try a little , besides your cigarette . __eou__ What ’ s wrong with that ? Cigarette is the thing I go crazy for . __eou__ Not for me , Dick . __eou__

Are things still going badly with your houseguest ? __eou__ Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . __eou__ Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . __eou__ You ’ re right . Everything is probably going to come to a head to

# Load twitter dataset

In [108]:
#Our text dataset
corpus_name = "dailydialog"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding="utf-8") as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "full.txt"))

@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.	@sprintcare and how do you propose we do that

@sprintcare I have sent several private messages and no one is responding as usual	@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.

@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.	@sprintcare I have sent several private messages and no one is responding as usual

@sprintcare I did.	@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.

@115712 Can you please send us a private message, so that I can gain further details about your account?	@sprintcare I did.

@sprintcare You gonna magically change your connectivity for me and my whole family ? 🤥 💯	@115713 This is saddening to hear. Please shoot us a DM,

## Create formatted data file

We'll create a formatted data file in which each line contains a tab-separated query sentence and a response sentence pair.

The following functions pase the `dailogues_text.txt` data file.
- `loadLines` splits each line of the file into conversations
- `extractSentencePairs` extracts pairs of sentences from conversations

In [111]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName):
    conversations = []
    with open(fileName, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split("__eou__")
            conversations.append(values)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation[i].strip()
            targetLine = conversation[i+1].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

Now we call the above functions to create a new file : `formatted_dialogues_text.txt`

In [112]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_dialogues_text.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("\nLoading conversations...")
conversations = loadLines(os.path.join(corpus, "dialogues_text.txt"))

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Loading conversations...

Writing newly formatted file...

Sample lines from file:
The kitchen stinks .	I'll throw out the garbage .

So Dick , how about getting some coffee for tonight ?	Coffee ? I don ’ t honestly like that kind of stuff .

Coffee ? I don ’ t honestly like that kind of stuff .	Come on , you can at least try a little , besides your cigarette .

Come on , you can at least try a little , besides your cigarette .	What ’ s wrong with that ? Cigarette is the thing I go crazy for .

What ’ s wrong with that ? Cigarette is the thing I go crazy for .	Not for me , Dick .

Are things still going badly with your houseguest ?	Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw .

Getting worse . Now he ’ s eating me out of 

# For twitter dataset

In [ ]:
#### split our dataset
def loadLines(fileName):
    conversations = []
    with open(fileName, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split("\t")
            conversations.append(values)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation[i].strip()
            targetLine = conversation[i+1].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

# Define path to new file
datafile = os.path.join(corpus, "formatted_twittertext.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("\nLoading conversations...")
conversations = loadLines(os.path.join(corpus, "full.txt"))

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)

## Load and trim data

Now let's create a vocabulary and load query/response sentence pairs into memory.

First we must create a mapping of each word to a discrete numerical space (the index value).

Voc class keeps the mapping from words to indexes, a reverse mapping of indexes to words, a count of each word and a total word count.
There are 3 central methods :
- `addWord` to add a word to the vocabulary
- `addSentence` to add all words in a sentence
- `trim` for trimming infrequently seen words

In [113]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Before assemble our vocabulary and query/response sentence pairs we must perform some preprocessing.

1. Convert the Unicode strings to ASCII with `unicodeToAscii`.
2. Convert all letters to lowercase and trim all non-letter characters except basic punctuation `normalizeString`
3. Filter out sentences witg length greater than the `MAX_LENGTH` threshold in `filterPairs`


In [114]:
#handle dull_responses now
dull_responses = ["I do not know what you are talking about.", "I do not know.", 
 "You do not know.", "You know what I mean.", "I know what you mean.", 
 "You know what I am saying.", "You do not know anything."]

In [115]:
MAX_LENGTH = 15  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters(replace them using space)
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for d in dull_responses:
        voc.addSentence(d)
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

Finally assmble voc and pairs

In [117]:
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 89862 sentence pairs
Trimmed to 43724 sentence pairs
Counting words...
Counted words: 10126

pairs:
['the kitchen stinks .', 'i ll throw out the garbage .']
['so dick how about getting some coffee for tonight ?', 'coffee ? i don t honestly like that kind of stuff .']
['coffee ? i don t honestly like that kind of stuff .', 'come on you can at least try a little besides your cigarette .']
['would you mind waiting a while ?', 'well how long will it be ?']
['i swear i m going to kill you for this .', 'what s wrong ? didn t you think it was fun ? !']
['never ! but thank you for inviting me .', 'come on . you ll feel better after we hit the showers .']
['certainly . how about spaghetti with clams and shrimps .', 'sounds delicious . ok . she ll try that .']
['can you manage chopsticks ?', 'why not ? see .']
['why not ? see .', 'good mastery . how do you like our chinese food ?']
['i m exhausted .', 'okay let s go home .']


In [118]:
voc.num_words

10126

### Trimming rarely used words out of vocab

One tactic beneficial to achieve faster convergence during training is trimming rarely used words out of our vocabulary.

1. Trim words used under `MIN_COUNT` threshold using `voc.trim`
2. Filter out pairs with trimmed words

In [119]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 6159 / 10123 = 0.6084
Trimmed from 43724 pairs to 38716, 0.8855 of total


# Divide the dataset into training set and testing set

In [150]:
#Separate the dataset to train and test

test_index = np.random.choice(a=len(pairs), size=100, replace=False, p=None) #the size of testing set is 100 here
test_pairs = []
for j in test_index:
    test_pairs.append(pairs[j])

full_index = np.arange(len(pairs))


#build the train pairs

train_index = (list(set(full_index).difference(set(test_index))))
train_pairs = []
for i in train_index:
    train_pairs.append(pairs[i])

#generate test_questions
test_questions = []
for k in test_pairs:
    test_questions.append(k[0])
print(len(test_questions))

100
38616
100


In [151]:
# We set use the train_pairs to train!!
pairs = train_pairs

## Prepare Data for Models

BATCH TECHNIQUE

To accomodate sentences of different sizes in the same batch we make our batched input tensor of shape `(max_length, batch_size)` where sentences shorter than the max_length are zeropadded after the `EOS_token`.

- `inputVar` function handles the process of converting sentences to tensor. It returns a tensor of `lengths` for each sequence in the batch for the decoder.
- `outputVar` function performs the same as `inputVar` but returns a binary mask tensor and a maximum target sentence length.
- `batch2TrainData` takes a bunch of pairs and returns the input and target tensors

In [152]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  16,  130,   50,  484,  187],
        [  17,  148,    8,    9,   71],
        [ 139,    9,  688,    8,    2],
        [   8,  538,   76,   61,    0],
        [  62, 1627,   64,   30,    0],
        [ 762,  359, 1929,    2,    0],
        [ 203,   15,   30,    0,    0],
        [ 343,    2,    2,    0,    0],
        [  35,    0,    0,    0,    0],
        [  15,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([11,  8,  8,  6,  3])
target_variable: tensor([[  23,   23,  130, 2872,   95],
        [ 374,   55,   16,  181,    8],
        [ 203,  159,   50,  766,   42],
        [  30,   13,   15,   15, 1633],
        [   2, 1700,    2,    2, 2070],
        [   0,   30,    0,    0, 1974],
        [   0,    2,    0,    0,   15],
        [   0,    0,    0,    0,    2]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  Tr

## Seq2Seq

The brain of our chatbot is a sequence to sequence model.

One RNN acts as an _encoder_  which encodes a variable length input sequence to a fixed-length context vector (the final hidden layer of the RNN).
The second RNN is a _decoder_ which takes a s input a word and a context vector and returns a guess for the next word in the sequence.

 ### Encoder

The encoder RNN iterates through tokens and outputs an "output" vector and a "hidden sate" vector. the hidden state vector is passed to the next time step while the output vector is recorder.
The encoder transforms the context it saw at each point in the sequence into a set of points in a high dimensional space. The decoder will use it to generate the outputted word.

We use a bidirectional multi-layered Gated Recurrent Unit.
It gives the advantage of encoding both past and future context !

Computation Graph :
1. Convert word indexed to embeddings
2. Pack padded batch of sequences for RNN module
3. Forward pass through GRU
4. Unpack padding
5. Sum bidirectional GRU outputs
6. Return output and final hidden state

In [153]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

### Decoder

The decoder RNN uses the encoder's context vectors and internal hidden states to generate the next word of the sequence.
It continues generating words until an `EOS_token`.
The problem with a vanilla seq2seq decoder is that if we rely woley on the context vector it will have information loss. (especially with long input sequences).

-> `attention mechanism` allows the decoder to pay attention to certain parts of the input sequence.

In [154]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

Now that the attention submodule is implemented let's dive into the actual decoder model.

Computation Graph:
1. Get embedding of current input word.
2. Forward through unidirectional GRU
3. Calculate attention weights from the current GRU output
4. Multiply attention weights to encoder outputs to get a new context vector
5. Concatenate weighted context vector and GRU using Luong eq
6. Predict next word
7. Return output and final hidden state

In [155]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## Training 

`maskNNLLLoss` calculates the average negative log likelihood of the elements that correspond to a 1 in the mask tensor.

In [156]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Single training iteration (single batch of inputs)

Couple of clever tricks :
- `teacher forcing` at some probability (set by `teacher_forcing_ratio`) current target word is used as the decoder's next input rather than using the decoder's current guess.
- `gradient clipping` commonly used technique for countering the "exploding gradient" problem.

Sequence of operations :
1. Forward pass entire input batch through encoder
2. initialize decoder inputs as SOS_token and hidden state as the encoder's final hidden state
3. Forward input batch sequence through decoder one time step at a time
4. If teacher forcing : set next decoder input as the current target else : set next decoder input as the current decoder ouptput
5. Calculate and accumulate loss
6. Perform backpropagation
7. Clip gradients
8. Update encoder and decoder

In [157]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

### Training iterations + save our model to run inferences or continue training !

In [158]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## Gready decoding for generating sentences

Decoding method when training is not using teacher forcing. At each time step we choose the word from decoder_output with the highest softmax value.

In [159]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

## Evaluation of model

In [160]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

## Run model

possible to laod from checkpoint

In [161]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 10000 # 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))
#print(loadFilename)

# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    #checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


## Train !

In [164]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 5000 #4000
print_every = 1
save_every = 1000

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 1.6210
Iteration: 2; Percent complete: 0.0%; Average loss: 1.6673
Iteration: 3; Percent complete: 0.1%; Average loss: 1.5388
Iteration: 4; Percent complete: 0.1%; Average loss: 1.7847
Iteration: 5; Percent complete: 0.1%; Average loss: 1.6770
Iteration: 6; Percent complete: 0.1%; Average loss: 1.7592
Iteration: 7; Percent complete: 0.1%; Average loss: 1.6562
Iteration: 8; Percent complete: 0.2%; Average loss: 1.6681
Iteration: 9; Percent complete: 0.2%; Average loss: 1.7437
Iteration: 10; Percent complete: 0.2%; Average loss: 1.6331
Iteration: 11; Percent complete: 0.2%; Average loss: 1.5204
Iteration: 12; Percent complete: 0.2%; Average loss: 1.5464
Iteration: 13; Percent complete: 0.3%; Average loss: 1.7947
Iteration: 14; Percent complete: 0.3%; Average loss: 1.5542
Iteration: 15; Percent complete: 0.3%; Average loss: 1.6107
Iteration: 16; Percent complete: 0.3%

Iteration: 136; Percent complete: 2.7%; Average loss: 1.7878
Iteration: 137; Percent complete: 2.7%; Average loss: 1.4713
Iteration: 138; Percent complete: 2.8%; Average loss: 1.6263
Iteration: 139; Percent complete: 2.8%; Average loss: 1.5842
Iteration: 140; Percent complete: 2.8%; Average loss: 1.6343
Iteration: 141; Percent complete: 2.8%; Average loss: 1.8690
Iteration: 142; Percent complete: 2.8%; Average loss: 1.6401
Iteration: 143; Percent complete: 2.9%; Average loss: 1.6814
Iteration: 144; Percent complete: 2.9%; Average loss: 1.5768
Iteration: 145; Percent complete: 2.9%; Average loss: 1.6413
Iteration: 146; Percent complete: 2.9%; Average loss: 1.5304
Iteration: 147; Percent complete: 2.9%; Average loss: 1.6444
Iteration: 148; Percent complete: 3.0%; Average loss: 1.7518
Iteration: 149; Percent complete: 3.0%; Average loss: 1.6963
Iteration: 150; Percent complete: 3.0%; Average loss: 1.5927
Iteration: 151; Percent complete: 3.0%; Average loss: 1.9397
Iteration: 152; Percent 

Iteration: 275; Percent complete: 5.5%; Average loss: 1.6373
Iteration: 276; Percent complete: 5.5%; Average loss: 1.4388
Iteration: 277; Percent complete: 5.5%; Average loss: 1.6430
Iteration: 278; Percent complete: 5.6%; Average loss: 1.7255
Iteration: 279; Percent complete: 5.6%; Average loss: 1.7352
Iteration: 280; Percent complete: 5.6%; Average loss: 1.7023
Iteration: 281; Percent complete: 5.6%; Average loss: 1.4688
Iteration: 282; Percent complete: 5.6%; Average loss: 1.7460
Iteration: 283; Percent complete: 5.7%; Average loss: 1.6303
Iteration: 284; Percent complete: 5.7%; Average loss: 1.6022
Iteration: 285; Percent complete: 5.7%; Average loss: 1.5300
Iteration: 286; Percent complete: 5.7%; Average loss: 1.6653
Iteration: 287; Percent complete: 5.7%; Average loss: 1.6381
Iteration: 288; Percent complete: 5.8%; Average loss: 1.5474
Iteration: 289; Percent complete: 5.8%; Average loss: 1.7046
Iteration: 290; Percent complete: 5.8%; Average loss: 1.5676
Iteration: 291; Percent 

Iteration: 414; Percent complete: 8.3%; Average loss: 1.5683
Iteration: 415; Percent complete: 8.3%; Average loss: 1.4507
Iteration: 416; Percent complete: 8.3%; Average loss: 1.6693
Iteration: 417; Percent complete: 8.3%; Average loss: 1.8517
Iteration: 418; Percent complete: 8.4%; Average loss: 1.6954
Iteration: 419; Percent complete: 8.4%; Average loss: 1.6007
Iteration: 420; Percent complete: 8.4%; Average loss: 1.6230
Iteration: 421; Percent complete: 8.4%; Average loss: 1.5706
Iteration: 422; Percent complete: 8.4%; Average loss: 1.4830
Iteration: 423; Percent complete: 8.5%; Average loss: 1.6568
Iteration: 424; Percent complete: 8.5%; Average loss: 1.3194
Iteration: 425; Percent complete: 8.5%; Average loss: 1.4961
Iteration: 426; Percent complete: 8.5%; Average loss: 1.5952
Iteration: 427; Percent complete: 8.5%; Average loss: 1.7483
Iteration: 428; Percent complete: 8.6%; Average loss: 1.7489
Iteration: 429; Percent complete: 8.6%; Average loss: 1.4735
Iteration: 430; Percent 

Iteration: 548; Percent complete: 11.0%; Average loss: 1.6018
Iteration: 549; Percent complete: 11.0%; Average loss: 1.6572
Iteration: 550; Percent complete: 11.0%; Average loss: 1.6201
Iteration: 551; Percent complete: 11.0%; Average loss: 1.6077
Iteration: 552; Percent complete: 11.0%; Average loss: 1.6433
Iteration: 553; Percent complete: 11.1%; Average loss: 1.5892
Iteration: 554; Percent complete: 11.1%; Average loss: 1.3386
Iteration: 555; Percent complete: 11.1%; Average loss: 1.5183
Iteration: 556; Percent complete: 11.1%; Average loss: 1.5604
Iteration: 557; Percent complete: 11.1%; Average loss: 1.5527
Iteration: 558; Percent complete: 11.2%; Average loss: 1.5158
Iteration: 559; Percent complete: 11.2%; Average loss: 1.6135
Iteration: 560; Percent complete: 11.2%; Average loss: 1.6686
Iteration: 561; Percent complete: 11.2%; Average loss: 1.4411
Iteration: 562; Percent complete: 11.2%; Average loss: 1.4768
Iteration: 563; Percent complete: 11.3%; Average loss: 1.4221
Iteratio

...

Iteration: 4399; Percent complete: 88.0%; Average loss: 0.7836
Iteration: 4400; Percent complete: 88.0%; Average loss: 0.7989
Iteration: 4401; Percent complete: 88.0%; Average loss: 0.7607
Iteration: 4402; Percent complete: 88.0%; Average loss: 0.8934
Iteration: 4403; Percent complete: 88.1%; Average loss: 0.6151
Iteration: 4404; Percent complete: 88.1%; Average loss: 0.6527
Iteration: 4405; Percent complete: 88.1%; Average loss: 0.8471
Iteration: 4406; Percent complete: 88.1%; Average loss: 0.5958
Iteration: 4407; Percent complete: 88.1%; Average loss: 0.8613
Iteration: 4408; Percent complete: 88.2%; Average loss: 0.7512
Iteration: 4409; Percent complete: 88.2%; Average loss: 0.6893
Iteration: 4410; Percent complete: 88.2%; Average loss: 0.6725
Iteration: 4411; Percent complete: 88.2%; Average loss: 0.7762
Iteration: 4412; Percent complete: 88.2%; Average loss: 0.6644
Iteration: 4413; Percent complete: 88.3%; Average loss: 0.7405
Iteration: 4414; Percent complete: 88.3%; Average loss:

Iteration: 4533; Percent complete: 90.7%; Average loss: 0.6900
Iteration: 4534; Percent complete: 90.7%; Average loss: 0.6357
Iteration: 4535; Percent complete: 90.7%; Average loss: 0.6065
Iteration: 4536; Percent complete: 90.7%; Average loss: 0.6540
Iteration: 4537; Percent complete: 90.7%; Average loss: 0.6288
Iteration: 4538; Percent complete: 90.8%; Average loss: 0.7615
Iteration: 4539; Percent complete: 90.8%; Average loss: 0.6739
Iteration: 4540; Percent complete: 90.8%; Average loss: 0.6103
Iteration: 4541; Percent complete: 90.8%; Average loss: 0.6990
Iteration: 4542; Percent complete: 90.8%; Average loss: 0.7351
Iteration: 4543; Percent complete: 90.9%; Average loss: 0.6591
Iteration: 4544; Percent complete: 90.9%; Average loss: 0.8146
Iteration: 4545; Percent complete: 90.9%; Average loss: 0.6814
Iteration: 4546; Percent complete: 90.9%; Average loss: 0.6425
Iteration: 4547; Percent complete: 90.9%; Average loss: 0.6770
Iteration: 4548; Percent complete: 91.0%; Average loss:

Iteration: 4665; Percent complete: 93.3%; Average loss: 0.6796
Iteration: 4666; Percent complete: 93.3%; Average loss: 0.6547
Iteration: 4667; Percent complete: 93.3%; Average loss: 0.7971
Iteration: 4668; Percent complete: 93.4%; Average loss: 0.7100
Iteration: 4669; Percent complete: 93.4%; Average loss: 0.6441
Iteration: 4670; Percent complete: 93.4%; Average loss: 0.8280
Iteration: 4671; Percent complete: 93.4%; Average loss: 0.6438
Iteration: 4672; Percent complete: 93.4%; Average loss: 0.5908
Iteration: 4673; Percent complete: 93.5%; Average loss: 0.7176
Iteration: 4674; Percent complete: 93.5%; Average loss: 0.8058
Iteration: 4675; Percent complete: 93.5%; Average loss: 0.6697
Iteration: 4676; Percent complete: 93.5%; Average loss: 0.6789
Iteration: 4677; Percent complete: 93.5%; Average loss: 0.6868
Iteration: 4678; Percent complete: 93.6%; Average loss: 0.8092
Iteration: 4679; Percent complete: 93.6%; Average loss: 0.6088
Iteration: 4680; Percent complete: 93.6%; Average loss:

Iteration: 4798; Percent complete: 96.0%; Average loss: 0.6554
Iteration: 4799; Percent complete: 96.0%; Average loss: 0.6873
Iteration: 4800; Percent complete: 96.0%; Average loss: 0.7473
Iteration: 4801; Percent complete: 96.0%; Average loss: 0.7209
Iteration: 4802; Percent complete: 96.0%; Average loss: 0.6174
Iteration: 4803; Percent complete: 96.1%; Average loss: 0.6743
Iteration: 4804; Percent complete: 96.1%; Average loss: 0.6538
Iteration: 4805; Percent complete: 96.1%; Average loss: 0.7159
Iteration: 4806; Percent complete: 96.1%; Average loss: 0.6986
Iteration: 4807; Percent complete: 96.1%; Average loss: 0.6225
Iteration: 4808; Percent complete: 96.2%; Average loss: 0.6727
Iteration: 4809; Percent complete: 96.2%; Average loss: 0.5896
Iteration: 4810; Percent complete: 96.2%; Average loss: 0.5590
Iteration: 4811; Percent complete: 96.2%; Average loss: 0.7209
Iteration: 4812; Percent complete: 96.2%; Average loss: 0.7008
Iteration: 4813; Percent complete: 96.3%; Average loss:

Iteration: 4933; Percent complete: 98.7%; Average loss: 0.4858
Iteration: 4934; Percent complete: 98.7%; Average loss: 0.5641
Iteration: 4935; Percent complete: 98.7%; Average loss: 0.5754
Iteration: 4936; Percent complete: 98.7%; Average loss: 0.7496
Iteration: 4937; Percent complete: 98.7%; Average loss: 0.6565
Iteration: 4938; Percent complete: 98.8%; Average loss: 0.6601
Iteration: 4939; Percent complete: 98.8%; Average loss: 0.6772
Iteration: 4940; Percent complete: 98.8%; Average loss: 0.5617
Iteration: 4941; Percent complete: 98.8%; Average loss: 0.7373
Iteration: 4942; Percent complete: 98.8%; Average loss: 0.7219
Iteration: 4943; Percent complete: 98.9%; Average loss: 0.7000
Iteration: 4944; Percent complete: 98.9%; Average loss: 0.7596
Iteration: 4945; Percent complete: 98.9%; Average loss: 0.6640
Iteration: 4946; Percent complete: 98.9%; Average loss: 0.6497
Iteration: 4947; Percent complete: 98.9%; Average loss: 0.7084
Iteration: 4948; Percent complete: 99.0%; Average loss:

## RUN AN PLAY(one question get one answer)

In [166]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()
#encoder.train()
#decoder.train()
# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting
evaluateInput(encoder, decoder, searcher, voc)

> Hello!
Bot: hi lucy are you coming to church ? ! ! !
> q


# Generate two chatbots conversation(20 turns)

In [167]:
def evaluate_con(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput_con(encoder, decoder, searcher, voc):
    input_sentence = input()
    for i in range(20):
            # Get input sentence                
            print('Bot1:', input_sentence)
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot2:', ' '.join(output_words))
            output_sentence = ' '.join(output_words)
                
            output_sentence = normalizeString(output_sentence)
                
            input_words = evaluate_con(encoder, decoder, searcher, voc, output_sentence)
                
            input_words[:] = [x for x in input_words if not (x == 'EOS' or x == 'PAD')]
                
            input_sentence = ' '.join(input_words)
                
                

In [168]:
#generate conversations
encoder.eval()
decoder.eval()
#encoder.train()
#decoder.train()
# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting
evaluateInput_con(encoder, decoder, searcher, voc)

Bot1: Hello!
Bot2: hi lucy are you coming to church ? ! ! !
Bot1: not yet . by the way the table has already been bothering ! .
Bot2: we ve been really busy . since we need to finish our company .
Bot1: great . i really appreciate your help . ! ! !
Bot2: thank you ! ! ! ! ! ! !
Bot1: and merry christmas ! ! ! ! ! !
Bot2: merry christmas ! ! ! ! ! ! !
Bot1: so merry christmas ! ! ! ! ! !
Bot2: so so so so all my coffee are coming to the house ? !
Bot1: certainly . i haven t done anything for you . ! !
Bot2: i d like to talk to you about your daughter . ! !
Bot1: is that right ? ? ! ? ! !
Bot2: yes . when is the next class ? ! ! !
Bot1: next june is next to the library . first ! ! !
Bot2: thank you ! ! ! ! ! ! !
Bot1: and merry christmas ! ! ! ! ! !
Bot2: merry christmas ! ! ! ! ! ! !
Bot1: so merry christmas ! ! ! ! ! !
Bot2: so so so so all my coffee are coming to the house ? !
Bot1: certainly . i haven t done anything for you . ! !
Bot2: i d like to talk to you about your daughter . !

## simulate the conversation turns

In [182]:
def evaluateInput_sim(encoder, decoder, searcher, voc):
    #input_sentence = input()
    turns_count = []
    for question in test_questions:
        input_sentence = question
        turns = 0
        for i in range(10):
                # Get input sentence   
                turns += 1
            
                check_pairs = ['','']
            
                #print('Bot1:', input_sentence)

                if input_sentence == 'q' or input_sentence == 'quit': break

                input_sentence = normalizeString(input_sentence)
            
                check_pairs[0] = input_sentence
            
                output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)

                output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            
            
                #print('Bot2:', ' '.join(output_words))
                output_sentence = ' '.join(output_words)
                
                output_sentence = normalizeString(output_sentence)
                
                input_words = evaluate(encoder, decoder, searcher, voc, output_sentence)
                
                input_words[:] = [x for x in input_words if not (x == 'EOS' or x == 'PAD')]
                
                input_sentence = ' '.join(input_words)
            
                check_pairs[1] = normalizeString(input_sentence)
            
                model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

                #Compute embedding for both lists
                embedding_1 = model.encode(check_pairs[0], convert_to_tensor=True)
                embedding_2 = model.encode(check_pairs[1], convert_to_tensor=True)

                overlap_score = util.pytorch_cos_sim(embedding_1, embedding_2)
            
                #print(overlap_score)
            
                if overlap_score > 0.5:
                        break
        print('turns:', turns)
        turns_count.append(turns)
    
    print('The average number of simulation turns:', np.mean(turns_count))
                
            
            

In [183]:
#generate conversations
encoder.eval()
decoder.eval()
#encoder.train()
#decoder.train()
# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting
evaluateInput_sim(encoder, decoder, searcher, voc)

turns: 10
turns: 10
turns: 6
turns: 9
turns: 10
turns: 4
turns: 5
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 2
turns: 10
turns: 10
turns: 10
turns: 10
turns: 3
turns: 10
turns: 10
turns: 7
turns: 10
turns: 10
turns: 8
turns: 5
turns: 10
turns: 10
turns: 3
turns: 10
turns: 10
turns: 10
turns: 4
turns: 10
turns: 10
turns: 5
turns: 10
turns: 10
turns: 10
turns: 6
turns: 10
turns: 8
turns: 9
turns: 2
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 4
turns: 5
turns: 10
turns: 10
turns: 6
turns: 10
turns: 10
turns: 1
turns: 10
turns: 3
turns: 10
turns: 10
turns: 3
turns: 2
turns: 7
turns: 10
turns: 9
turns: 6
turns: 4
turns: 10
turns: 10
turns: 2
turns: 10
turns: 4
turns: 10
turns: 10
turns: 4
turns: 3
turns: 10
turns: 10
turns: 2
turns: 5
turns: 10
turns: 10
turns: 3
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 10
turns: 7
The average number of simulation tu

## Generate responses

In [204]:
def generate_responses(encoder, decoder, searcher, voc):
    test_responses = []
    for question in test_questions:
        input_sentence = question
        if input_sentence == 'q' or input_sentence == 'quit': break
        input_sentence = normalizeString(input_sentence)
        output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
        output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
        output_sentence = ' '.join(output_words)
        test_responses.append(output_sentence)
    return test_responses

In [205]:
#generate conversations
encoder.eval()
decoder.eval()
#encoder.train()
#decoder.train()
# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting
test_responses = generate_responses(encoder, decoder, searcher, voc)

## Another try of ngram diversity

In [208]:
class Unigram:
    def __init__(self, sentences):
        self.unigram_frequencies = dict()
        self.corpus_length = 0
        for sentence in sentences:
            sentence = normalizeString(sentence)
            sen = sentence.split()
            for word in sen:
                self.unigram_frequencies[word] = self.unigram_frequencies.get(word, 0) + 1
                self.corpus_length += 1
        self.unique_words = len(self.unigram_frequencies)

def calculate_number_of_unigrams(sentences):
    unigram_count = 0
    for sentence in sentences:
        sentence = normalizeString(sentence)
        sent = sentence.split()
        unigram_count += len(sent)
    return unigram_count

unigram_count = calculate_number_of_unigrams(test_responses)
Uni = Unigram(test_responses)
print(Uni.unigram_frequencies)
print('distinct unigram number:',Uni.unique_words)
print('number of unigram:', unigram_count)
print('ratio of distinct unigram:', Uni.unique_words/unigram_count)

{'yes': 14, 'i': 46, 'm': 11, 'looking': 1, 'for': 4, 'a': 31, 'house': 2, '.': 192, 'next': 8, 'friday': 2, '!': 119, 'so': 1, 'what': 10, '?': 39, 'that': 11, 's': 21, 'great': 4, 'was': 9, 'the': 22, 'score': 1, 'last': 3, 'month': 2, 'again': 8, 'are': 19, 'broken': 1, 'have': 10, 'my': 15, 'own': 2, 'car': 4, 'week': 3, 'let': 2, 'me': 4, 'see': 5, 'doctor': 1, 'time': 4, 'you': 35, 'room': 2, 'perfect': 1, 'like': 8, 'color': 1, 'do': 12, 'think': 6, 'school': 1, 'will': 7, 'be': 8, 'here': 4, 'bar': 1, 'of': 9, 'western': 1, 'it': 17, 'fun': 1, 'really': 3, 'good': 5, 'when': 3, 'say': 1, 'yeah': 2, 'too': 2, 'visiting': 1, 'other': 4, 'times': 1, 'on': 4, 'sale': 1, 'just': 3, 'moment': 1, 'would': 3, 'to': 19, 'look': 2, 'at': 6, 'some': 5, 'your': 4, 'products': 1, 'well': 3, 'little': 1, 'easier': 1, 'know': 2, 'work': 2, 'founded': 1, 'in': 14, 'about': 2, 'six': 1, 'o': 2, 'clock': 2, 'serious': 1, 'going': 6, 'right': 4, 'choice': 1, 'am': 5, 'hurry': 2, 'doing': 2, 'lapt

In [211]:
class Bigram(Unigram):
    def __init__(self, sentences):
        Unigram.__init__(self, sentences)
        self.bigram_frequencies = dict()
        for sentence in sentences:
            sentence = normalizeString(sentence)
            sen = sentence.split()
            previous_word = None
            for word in sen:
                if previous_word != None:
                    self.bigram_frequencies[(previous_word, word)] = self.bigram_frequencies.get((previous_word, word),
                                                                                                 0) + 1
                previous_word = word
        self.unique__bigram_words = len(self.unigram_frequencies)

def calculate_number_of_bigrams(sentences):
    bigram_count = 0
    for sentence in sentences:
        sentence = normalizeString(sentence)
        sent = sentence.split()
        bigram_count += len(sent) - 1
    return bigram_count

bigram_count = calculate_number_of_bigrams(test_responses)
Bi = Bigram(test_responses)
print(Bi.bigram_frequencies)
print('distinct bigram number:',Bi.unique__bigram_words)
print('number of bigram:', bigram_count)
print('ratio of distinct bigram:', Bi.unique__bigram_words/bigram_count)

{('yes', 'i'): 7, ('i', 'm'): 11, ('m', 'looking'): 1, ('looking', 'for'): 1, ('for', 'a'): 2, ('a', 'house'): 1, ('house', '.'): 2, ('.', 'next'): 5, ('next', 'friday'): 2, ('friday', '.'): 2, ('.', '!'): 50, ('so', 'what'): 1, ('what', '?'): 2, ('?', '.'): 5, ('!', '!'): 56, ('that', 's'): 8, ('s', 'great'): 2, ('great', '!'): 1, ('!', '.'): 3, ('what', 'was'): 1, ('was', 'the'): 2, ('the', 'score'): 1, ('score', '?'): 1, ('?', 'last'): 1, ('last', 'a'): 1, ('a', 'month'): 1, ('month', '.'): 2, ('.', 'again'): 6, ('again', '.'): 1, ('yes', '.'): 5, ('.', 'are'): 11, ('are', 'broken'): 1, ('broken', '.'): 1, ('.', '.'): 38, ('i', 'have'): 3, ('have', 'my'): 1, ('my', 'own'): 2, ('own', 'car'): 1, ('car', '.'): 1, ('.', 'last'): 1, ('last', 'week'): 1, ('week', '.'): 2, ('let', 'me'): 2, ('me', 'see'): 1, ('see', 'a'): 1, ('a', 'doctor'): 1, ('doctor', '.'): 1, ('.', 'what'): 3, ('what', 'time'): 1, ('time', 'you'): 1, ('you', 'have'): 4, ('have', 'a'): 6, ('a', 'room'): 1, ('room', '.

# Add one more trigram diversity

In [213]:
class Trigram(Bigram):
    def __init__(self, sentences):
        Bigram.__init__(self, sentences)
        self.trigram_frequencies = dict()
        for sentence in sentences:
            sentence = normalizeString(sentence)
            sen = sentence.split()
            previous_word = None
            mid_word = None
            for word in sen:
                if previous_word != None:
                    self.trigram_frequencies[(previous_word, mid_word, word)] = self.trigram_frequencies.get((previous_word, mid_word, word), 0) + 1                   
                previous_word = mid_word
                mid_word = word
        self.unique__trigram_words = len(self.unigram_frequencies)

def calculate_number_of_trigrams(sentences):
        trigram_count = 0
        for sentence in sentences:
            sentence = normalizeString(sentence)
            sent = sentence.split()
            trigram_count += len(sent) - 2
        return trigram_count

trigram_count = calculate_number_of_trigrams(test_responses)
Tri = Trigram(test_responses)
print(Tri.trigram_frequencies)
print('distinct bigram number:',Tri.unique__trigram_words)
print('number of bigram:', trigram_count)
print('ratio of distinct bigram:', Tri.unique__trigram_words/trigram_count)

{('yes', 'i', 'm'): 1, ('i', 'm', 'looking'): 1, ('m', 'looking', 'for'): 1, ('looking', 'for', 'a'): 1, ('for', 'a', 'house'): 1, ('a', 'house', '.'): 1, ('house', '.', 'next'): 1, ('.', 'next', 'friday'): 2, ('next', 'friday', '.'): 2, ('friday', '.', '!'): 2, ('so', 'what', '?'): 1, ('what', '?', '.'): 1, ('?', '.', '!'): 3, ('.', '!', '!'): 27, ('!', '!', '!'): 26, ('that', 's', 'great'): 2, ('s', 'great', '!'): 1, ('great', '!', '.'): 1, ('!', '.', '!'): 3, ('what', 'was', 'the'): 1, ('was', 'the', 'score'): 1, ('the', 'score', '?'): 1, ('score', '?', 'last'): 1, ('?', 'last', 'a'): 1, ('last', 'a', 'month'): 1, ('a', 'month', '.'): 1, ('month', '.', 'again'): 1, ('.', 'again', '.'): 1, ('again', '.', 'again'): 1, ('yes', '.', 'are'): 1, ('.', 'are', 'broken'): 1, ('are', 'broken', '.'): 1, ('broken', '.', '.'): 1, ('.', '.', '!'): 15, ('yes', 'i', 'have'): 1, ('i', 'have', 'my'): 1, ('have', 'my', 'own'): 1, ('my', 'own', 'car'): 1, ('own', 'car', '.'): 1, ('car', '.', 'last'): 1

## Reinforcement learning

### Forward and backward models 

In [ ]:
#forward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
#forward_decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
#forward_encoder = forward_encoder.to(device)
#forward_decoder = forward_decoder.to(device)

In [22]:
forward_encoder = encoder
forward_decoder = decoder
forward_encoder = forward_encoder.to(device)
forward_decoder = forward_decoder.to(device)

In [23]:
backward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
backward_decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
backward_encoder = backward_encoder.to(device)
backward_decoder = backward_decoder.to(device)

### Utility functions

In [24]:
def convertResponse(response):
    size1 = len(response)
    size2 = batch_size
    npRes = np.zeros((size1, size2), dtype=np.int64)
    npLengths = np.zeros(size2, dtype=np.int64)
    for i in range(size1):
        prov = response[i].cpu().numpy()
        for j in range(prov.size):
            npLengths[j] = npLengths[j] + 1
            if prov.size > 1:
                npRes[i][j] = prov[j]
            else:
                npRes[i][j] = prov 
    res = torch.from_numpy(npRes)
    lengths = torch.from_numpy(npLengths)
    return res, lengths

In [25]:
def convertTarget(target):
    size1 = len(target)
    size2 = batch_size
    npRes = np.zeros((size1, size2), dtype=np.int64)
    mask = np.zeros((size1, size2), dtype=np.bool_)
    npLengths = np.zeros(size2, dtype=np.int64)
    for i in range(size1):
        prov = target[i].cpu().numpy()
        for j in range(prov.size):
            npLengths[j] = npLengths[j] + 1
            if prov.size > 1:
                npRes[i][j] = prov[j]
            else:
                npRes[i][j] = prov 
                
            if npRes[i][j] > 0:
                mask[i][j] = True
            else:
                mask[i][j] = False
            
    res = torch.from_numpy(npRes)
    lengths = torch.from_numpy(npLengths)
    mask= torch.from_numpy(mask)
    max_target_len = torch.max(lengths) #.detach().numpy()
    return res, mask, max_target_len

In [26]:
def transformTensorToSameShapeAs(tensor, shape):
    size1, size2 = shape
    npNewT = np.zeros((size1, size2), dtype=np.int64)
    npNewMask = np.zeros((size1, size2), dtype=np.bool_)
    tensorSize1, tensorSize2 = tensor.size()
    for i in range(tensorSize1):
        for j in range(tensorSize2):
            npNewT[i][j] = tensor[i][j]
            npNewMask[i][j]= True
    return torch.from_numpy(npNewT), torch.from_numpy(npNewMask)

### Training step for a single iteration 

In [27]:
def RL(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio):
    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")
    
    #Initialize variables
    loss=0
    #print_losses = []
    response=[]
    
    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    
    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    
    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

     # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
     # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            #print_losses.append(mask_loss.item() * nTotal)
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            #print_losses.append(mask_loss.item() * nTotal)
            
            #ni or decoder_output
            response.append(topi)
            
    return loss, max_target_len, response

Let's define the rewards method for each action.

It is composed of 3 different types of rewards :
- `ease of answering` which is basically saying that a turn generated by a machineshould be easy to respond to. 
- `information flow` : We  want  the  agent  to  contribute new information at each turn to keep the dialogue moving and avoid repetitive sequences. 
- `semantic coherence` measures the adequacy of responses to avoid situations in which the generated replies are highly rewarded but are ungrammatical or not coherent. 

In [28]:
def easeOfAnswering(input_variable, lengths, dull_responses, mask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio):
    NS=len(dull_responses)
    r1=0
    for d in dull_responses:
        d, mask, max_target_len = outputVar(d, voc)
        newD, newMask = transformTensorToSameShapeAs(d, input_variable.size())
        #tar, mask, max_target_len = convertTarget(d)
        forward_loss, forward_len, _ = RL(input_variable, lengths, newD, newMask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio)
        # log (1/P(a|s)) = CE  --> log(P(a | s)) = - CE
        if forward_len > 0:
            r1 -= forward_loss / forward_len
    if len(dull_responses) > 0:
        r1 = r1 / NS
    return r1

In [29]:
def informationFlow(responses):
    r2=0
    if(len(responses) > 2):
        #2 representations obtained from the encoder for two consecutive turns pi and pi+1
        h_pi = responses[-3]
        h_pi1 = responses[-1]
        # length of the two vector might not match
        min_length = min(len(h_pi), len(h_pi+1))
        h_pi = h_pi[:min_length]
        h_pi1 = h_pi1[:min_length]
        #cosine similarity 
        #cos_sim = 1 - distance.cosine(h_pi, h_pi1)
        cos_sim = 1 - distance.cdist(h_pi.cpu().numpy(), h_pi1.cpu().numpy(), 'cosine')
        #Handle negative cos_sim
        if np.any(cos_sim <= 0):
            r2 = - cos_sim
        else:
            r2 = - np.log(cos_sim)
        r2 = np.mean(r2) #为啥这里还要求个mean
    return r2

In [30]:
def semanticCoherence(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio):
    #print("IN R3:")
    #print("Input_variable :", input_variable.shape)
    #print("Lengths :", lengths.shape)
    #print("Target_variable :", target_variable.shape)
    #print("Mask :", mask.shape)
    #print("Max_Target_Len :", max_target_len)
    r3 = 0
    forward_loss, forward_len, _ = RL(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
    ep_input, lengths_trans = convertResponse(target_variable)
    #print("ep_input :", ep_input.shape)
    #print("Lengths transformed :", lengths_trans.shape)
    ep_target, mask_trans, max_target_len_trans = convertTarget(input_variable)
    #print("ep_target :", ep_target.shape)
    #print("mask transformed :", mask_trans.shape)
    #print("max_target_len_trans :", max_target_len_trans)
    backward_loss, backward_len, _ = RL(ep_input, lengths_trans, ep_target, mask_trans, max_target_len_trans, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
    if forward_len > 0:
        r3 += forward_loss / forward_len
    if backward_len > 0:
        r3+= backward_loss / backward_len
    return r3

In [31]:
l1=0.25
l2=0.25
l3=0.5

In [32]:
def calculate_rewards(input_var, lengths, target_var, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio):
    #rewards per episode
    ep_rewards = []
    #indice of current episode
    ep_num = 1
    #list of responses
    responses = []
    #input of current episode
    ep_input = input_var
    #target of current episode
    ep_target = target_var
    
    #ep_num bounded -> to redefine (MEDIUM POST)
    while (ep_num <= 10):
        
        print(ep_num)
        #generate current response with the forward model
        _, _, curr_response = RL(ep_input, lengths, ep_target, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Break if :
        # 1 -> dull response
        # 2 -> response is less than MIN_LENGTH
        # 3 -> repetition ie curr_response in responses
        if(len(curr_response) < MIN_COUNT):# or (curr_response in dull_responses) or (curr_response in responses)):
            break #这里是为啥？？
            
            
        #We can add the response to responses list
        #curr_response = torch.LongTensor(curr_response).view(-1, 1)
        #transform curr_response size
        #target = torch.zeros(960, 1)
        #target[:15, :] = curr_response
        #curr_response = target
        #print(curr_response.size())
        #curr_response = torch.reshape(curr_response, (15, 64))
        #print(curr_response.size())
        #curr_response = curr_response.to(device)
        #responses.append(curr_response) 
        
        
        #Ease of answering
        r1 = easeOfAnswering(ep_input, lengths, dull_responses, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Information flow
        r2 = informationFlow(responses) #这里的reponse是啥
        
        #Semantic coherence
        r3 = semanticCoherence(ep_input, lengths, target_var, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
        
        #Final reward as a weighted sum of rewards
        r = l1*r1 + l2*r2 + l3*r3
        
        #Add the current reward to the list
        ep_rewards.append(r.detach().cpu().numpy()) #不让r参与backward propogation
        
        #We can add the response to responses list
        curr_response, lengths = convertResponse(curr_response)
        curr_response = curr_response.to(device)
        responses.append(curr_response)
        
        #Next input is the current response
        ep_input = curr_response
        #Next target -> dummy
        ep_target = torch.zeros(MAX_LENGTH,batch_size,dtype=torch.int64)
        #ep_target = torch.LongTensor(torch.LongTensor([0] * MAX_LENGTH)).view(-1, 1)
        ep_target = ep_target.to(device)
        
        #Turn off the teacher forcing  after first iteration -> dummy target
        teacher_forcing_ratio = 0
        ep_num +=1
        
    #Take the mean of the episodic rewards
    return np.mean(ep_rewards) if len(ep_rewards) > 0 else 0 

Dull responses 

In [33]:
dull_responses = ["i do not know what you are talking about.", "i do not know.", 
 "you do not know.", "you know what i mean.", "i know what you mean.", 
 "you know what i am saying.", "you do not know anything."]
#dull_responses = []

### Training RL loop

In [34]:
def trainingRLLoop(model_name, voc, pairs, batch_size, forward_encoder, forward_encoder_optimizer, forward_decoder, forward_decoder_optimizer, backward_encoder, backward_encoder_optimizer, backward_decoder, backward_decoder_optimizer,teacher_forcing_ratio, dull_responses, n_iteration, print_every, save_every, save_dir):
    
    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    
    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    
    
    #Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        print("Iteration", iteration)
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        ##MODIFS HERE
        # Zero gradients the optimizer
        forward_encoder_optimizer.zero_grad()
        forward_decoder_optimizer.zero_grad()
        
        backward_encoder_optimizer.zero_grad()
        backward_decoder_optimizer.zero_grad()
        
        #Forward
        forward_loss, forward_len, _ = RL(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Calculate reward
        reward = calculate_rewards(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
        
        #Update forward seq2seq with loss scaled by reward
        loss = forward_loss * reward
        
        loss.backward()
        forward_encoder_optimizer.step()
        forward_decoder_optimizer.step()

        # Run a training iteration with batch
        print_loss += loss / forward_len
        
        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0
            
        #SAVE CHECKPOINT TO DO
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name)#, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## RUN 

In [35]:
#Configure RL model

model_name='RL_model_seq'
n_iteration = 10000
print_every=100
save_every=500
learning_rate = 0.00001
decoder_learning_ratio = 5.0
teacher_forcing_ratio = 0.5

# Ensure dropout layers are in train mode
forward_encoder.train()
forward_decoder.train()

backward_encoder.train()
backward_decoder.train()

# Initialize optimizers
print('Building optimizers ...')
forward_encoder_optimizer = optim.Adam(forward_encoder.parameters(), lr=learning_rate)
forward_decoder_optimizer = optim.Adam(forward_decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
backward_encoder_optimizer = optim.Adam(backward_encoder.parameters(), lr=learning_rate)
backward_decoder_optimizer = optim.Adam(backward_decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# If you have cuda, configure cuda to call
for state in forward_encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in forward_decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
            
for state in backward_encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in backward_decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
            
# Run training iterations
print("Starting Training!")
trainingRLLoop(model_name, voc, pairs, batch_size, forward_encoder, forward_encoder_optimizer, forward_decoder, forward_decoder_optimizer, backward_encoder, backward_encoder_optimizer, backward_decoder, backward_decoder_optimizer, teacher_forcing_ratio, dull_responses, n_iteration, print_every, save_every, save_dir)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration 1
1
Iteration 2
1
Iteration 3
1
2
3
4
5
6
7
8
9
10
Iteration 4
1
Iteration 5
1
Iteration 6
1
2
3
4
5
6
7
8
9
10
Iteration 7
1
2
3
4
5
6
7
8
9
10
Iteration 8
1
Iteration 9
1
Iteration 10
1
2
3
4
5
6
7
8
9
10
Iteration 11
1
Iteration 12
1
Iteration 13
1
2
3
4
5
6
7
8
9
10
Iteration 14
1
Iteration 15
1
Iteration 16
1
2
3
4
5
6
7
8
9
10
Iteration 17
1
2
3
4
5
6
7
8
9
10
Iteration 18
1
Iteration 19
1
2
3
4
5
6
7
8
9
10
Iteration 20
1
2
3
4
5
6
7
8
9
10
Iteration 21
1
2
3
4
5
6
7
8
9
10
Iteration 22
1
2
3
4
5
6
7
8
9
10
Iteration 23
1
Iteration 24
1
2
3
4
5
6
7
8
9
10
Iteration 25
1
2
3
4
5
6
7
8
9
10
Iteration 26
1
Iteration 27
1
Iteration 28
1
Iteration 29
1
2
3
4
5
6
7
8
9
10
Iteration 30
1
2
3
4
5
6
7
8
9
10
Iteration 31
1
2
3
4
5
6
7
8
9
10
Iteration 32
1
2
3
4
5
6
7
8
9
10
Iteration 33
1
2
3
4
5
6
7
8
9
10
Iteration 34
1
Iteration 35
1
2
3
4
5
6
7
8
9
10
Iteration 36
1
2
3
4
5
6
7
8
9
10
Iteration 37
1
It

1
Iteration 322
1
Iteration 323
1
Iteration 324
1
Iteration 325
1
Iteration 326
1
2
3
4
5
6
7
8
9
10
Iteration 327
1
Iteration 328
1
2
3
4
5
6
7
8
9
10
Iteration 329
1
2
3
4
5
6
7
8
9
10
Iteration 330
1
2
3
4
5
6
7
8
9
10
Iteration 331
1
Iteration 332
1
2
3
4
5
6
7
8
9
10
Iteration 333
1
2
3
4
5
6
7
8
9
10
Iteration 334
1
Iteration 335
1
2
3
4
5
6
7
8
9
10
Iteration 336
1
2
3
4
5
6
7
8
9
10
Iteration 337
1
2
3
4
5
6
7
8
9
10
Iteration 338
1
2
3
4
5
6
7
8
9
10
Iteration 339
1
Iteration 340
1
2
3
4
5
6
7
8
9
10
Iteration 341
1
Iteration 342
1
2
3
4
5
6
7
8
9
10
Iteration 343
1
Iteration 344
1
Iteration 345
1
Iteration 346
1
Iteration 347
1
Iteration 348
1
2
3
4
5
6
7
8
9
10
Iteration 349
1
2
3
4
5
6
7
8
9
10
Iteration 350
1
2
3
4
5
6
7
8
9
10
Iteration 351
1
Iteration 352
1
Iteration 353
1
2
3
4
5
6
7
8
9
10
Iteration 354
1
Iteration 355
1
Iteration 356
1
Iteration 357
1
2
3
4
5
6
7
8
9
10
Iteration 358
1
2
3
4
5
6
7
8
9
10
Iteration 359
1
Iteration 360
1
Iteration 361
1
Iteration 362
1


2
3
4
5
6
7
8
9
10
Iteration 642
1
2
3
4
5
6
7
8
9
10
Iteration 643
1
Iteration 644
1
2
3
4
5
6
7
8
9
10
Iteration 645
1
Iteration 646
1
2
3
4
5
6
7
8
9
10
Iteration 647
1
Iteration 648
1
2
3
4
5
6
7
8
9
10
Iteration 649
1
Iteration 650
1
2
3
4
5
6
7
8
9
10
Iteration 651
1
Iteration 652
1
Iteration 653
1
2
3
4
5
6
7
8
9
10
Iteration 654
1
2
3
4
5
6
7
8
9
10
Iteration 655
1
2
3
4
5
6
7
8
9
10
Iteration 656
1
2
3
4
5
6
7
8
9
10
Iteration 657
1
Iteration 658
1
2
3
4
5
6
7
8
9
10
Iteration 659
1
Iteration 660
1
2
3
4
5
6
7
8
9
10
Iteration 661
1
Iteration 662
1
2
3
4
5
6
7
8
9
10
Iteration 663
1
2
3
4
5
6
7
8
9
10
Iteration 664
1
Iteration 665
1
Iteration 666
1
Iteration 667
1
2
3
4
5
6
7
8
9
10
Iteration 668
1
2
3
4
5
6
7
8
9
10
Iteration 669
1
Iteration 670
1
2
3
4
5
6
7
8
9
10
Iteration 671
1
2
3
4
5
6
7
8
9
10
Iteration 672
1
Iteration 673
1
2
3
4
5
6
7
8
9
10
Iteration 674
1
Iteration 675
1
Iteration 676
1
Iteration 677
1
2
3
4
5
6
7
8
9
10
Iteration 678
1
2
3
4
5
6
7
8
9
10
Iteration

8
9
10
Iteration 959
1
2
3
4
5
6
7
8
9
10
Iteration 960
1
Iteration 961
1
2
3
4
5
6
7
8
9
10
Iteration 962
1
2
3
4
5
6
7
8
9
10
Iteration 963
1
Iteration 964
1
2
3
4
5
6
7
8
9
10
Iteration 965
1
Iteration 966
1
Iteration 967
1
Iteration 968
1
2
3
4
5
6
7
8
9
10
Iteration 969
1
Iteration 970
1
Iteration 971
1
2
3
4
5
6
7
8
9
10
Iteration 972
1
Iteration 973
1
2
3
4
5
6
7
8
9
10
Iteration 974
1
2
3
4
5
6
7
8
9
10
Iteration 975
1
2
3
4
5
6
7
8
9
10
Iteration 976
1
Iteration 977
1
2
3
4
5
6
7
8
9
10
Iteration 978
1
Iteration 979
1
2
3
4
5
6
7
8
9
10
Iteration 980
1
2
3
4
5
6
7
8
9
10
Iteration 981
1
2
3
4
5
6
7
8
9
10
Iteration 982
1
2
3
4
5
6
7
8
9
10
Iteration 983
1
Iteration 984
1
2
3
4
5
6
7
8
9
10
Iteration 985
1
2
3
4
5
6
7
8
9
10
Iteration 986
1
Iteration 987
1
Iteration 988
1
Iteration 989
1
Iteration 990
1
2
3
4
5
6
7
8
9
10
Iteration 991
1
2
3
4
5
6
7
8
9
10
Iteration 992
1
Iteration 993
1
2
3
4
5
6
7
8
9
10
Iteration 994
1
2
3
4
5
6
7
8
9
10
Iteration 995
1
2
3
4
5
6
7
8
9
10
It

7
8
9
10
Iteration 1267
1
Iteration 1268
1
Iteration 1269
1
2
3
4
5
6
7
8
9
10
Iteration 1270
1
2
3
4
5
6
7
8
9
10
Iteration 1271
1
Iteration 1272
1
Iteration 1273
1
2
3
4
5
6
7
8
9
10
Iteration 1274
1
2
3
4
5
6
7
8
9
10
Iteration 1275
1
2
3
4
5
6
7
8
9
10
Iteration 1276
1
Iteration 1277
1
2
3
4
5
6
7
8
9
10
Iteration 1278
1
Iteration 1279
1
Iteration 1280
1
2
3
4
5
6
7
8
9
10
Iteration 1281
1
Iteration 1282
1
Iteration 1283
1
2
3
4
5
6
7
8
9
10
Iteration 1284
1
Iteration 1285
1
2
3
4
5
6
7
8
9
10
Iteration 1286
1
Iteration 1287
1
2
3
4
5
6
7
8
9
10
Iteration 1288
1
2
3
4
5
6
7
8
9
10
Iteration 1289
1
Iteration 1290
1
2
3
4
5
6
7
8
9
10
Iteration 1291
1
2
3
4
5
6
7
8
9
10
Iteration 1292
1
Iteration 1293
1
2
3
4
5
6
7
8
9
10
Iteration 1294
1
2
3
4
5
6
7
8
9
10
Iteration 1295
1
Iteration 1296
1
2
3
4
5
6
7
8
9
10
Iteration 1297
1
2
3
4
5
6
7
8
9
10
Iteration 1298
1
2
3
4
5
6
7
8
9
10
Iteration 1299
1
2
3
4
5
6
7
8
9
10
Iteration 1300
1
Iteration: 1300; Percent complete: 13.0%; Average lo

...

1
2
3
4
5
6
7
8
9
10
Iteration 8795
1
Iteration 8796
1
Iteration 8797
1
Iteration 8798
1
2
3
4
5
6
7
8
9
10
Iteration 8799
1
2
3
4
5
6
7
8
9
10
Iteration 8800
1
2
3
4
5
6
7
8
9
10
Iteration: 8800; Percent complete: 88.0%; Average loss: 3.6988
Iteration 8801
1
Iteration 8802
1
2
3
4
5
6
7
8
9
10
Iteration 8803
1
Iteration 8804
1
Iteration 8805
1
Iteration 8806
1
2
3
4
5
6
7
8
9
10
Iteration 8807
1
Iteration 8808
1
2
3
4
5
6
7
8
9
10
Iteration 8809
1
Iteration 8810
1
Iteration 8811
1
2
3
4
5
6
7
8
9
10
Iteration 8812
1
2
3
4
5
6
7
8
9
10
Iteration 8813
1
2
3
4
5
6
7
8
9
10
Iteration 8814
1
Iteration 8815
1
Iteration 8816
1
2
3
4
5
6
7
8
9
10
Iteration 8817
1
Iteration 8818
1
Iteration 8819
1
Iteration 8820
1
Iteration 8821
1
Iteration 8822
1
2
3
4
5
6
7
8
9
10
Iteration 8823
1
Iteration 8824
1
Iteration 8825
1
2
3
4
5
6
7
8
9
10
Iteration 8826
1
Iteration 8827
1
Iteration 8828
1
2
3
4
5
6
7
8
9
10
Iteration 8829
1
Iteration 8830
1
2
3
4
5
6
7
8
9
10
Iteration 8831
1
Iteration 8832
1
Iter

8
9
10
Iteration 9094
1
2
3
4
5
6
7
8
9
10
Iteration 9095
1
2
3
4
5
6
7
8
9
10
Iteration 9096
1
Iteration 9097
1
2
3
4
5
6
7
8
9
10
Iteration 9098
1
2
3
4
5
6
7
8
9
10
Iteration 9099
1
2
3
4
5
6
7
8
9
10
Iteration 9100
1
2
3
4
5
6
7
8
9
10
Iteration: 9100; Percent complete: 91.0%; Average loss: 3.6284
Iteration 9101
1
2
3
4
5
6
7
8
9
10
Iteration 9102
1
2
3
4
5
6
7
8
9
10
Iteration 9103
1
Iteration 9104
1
Iteration 9105
1
2
3
4
5
6
7
8
9
10
Iteration 9106
1
Iteration 9107
1
Iteration 9108
1
2
3
4
5
6
7
8
9
10
Iteration 9109
1
Iteration 9110
1
2
3
4
5
6
7
8
9
10
Iteration 9111
1
Iteration 9112
1
Iteration 9113
1
Iteration 9114
1
2
3
4
5
6
7
8
9
10
Iteration 9115
1
Iteration 9116
1
2
3
4
5
6
7
8
9
10
Iteration 9117
1
2
3
4
5
6
7
8
9
10
Iteration 9118
1
Iteration 9119
1
Iteration 9120
1
2
3
4
5
6
7
8
9
10
Iteration 9121
1
Iteration 9122
1
2
3
4
5
6
7
8
9
10
Iteration 9123
1
Iteration 9124
1
2
3
4
5
6
7
8
9
10
Iteration 9125
1
Iteration 9126
1
Iteration 9127
1
2
3
4
5
6
7
8
9
10
Iteration 

10
Iteration 9388
1
2
3
4
5
6
7
8
9
10
Iteration 9389
1
2
3
4
5
6
7
8
9
10
Iteration 9390
1
Iteration 9391
1
2
3
4
5
6
7
8
9
10
Iteration 9392
1
Iteration 9393
1
Iteration 9394
1
Iteration 9395
1
Iteration 9396
1
2
3
4
5
6
7
8
9
10
Iteration 9397
1
Iteration 9398
1
Iteration 9399
1
Iteration 9400
1
2
3
4
5
6
7
8
9
10
Iteration: 9400; Percent complete: 94.0%; Average loss: 4.0303
Iteration 9401
1
Iteration 9402
1
2
3
4
5
6
7
8
9
10
Iteration 9403
1
Iteration 9404
1
Iteration 9405
1
2
3
4
5
6
7
8
9
10
Iteration 9406
1
Iteration 9407
1
Iteration 9408
1
Iteration 9409
1
2
3
4
5
6
7
8
9
10
Iteration 9410
1
2
3
4
5
6
7
8
9
10
Iteration 9411
1
Iteration 9412
1
2
3
4
5
6
7
8
9
10
Iteration 9413
1
Iteration 9414
1
Iteration 9415
1
2
3
4
5
6
7
8
9
10
Iteration 9416
1
Iteration 9417
1
2
3
4
5
6
7
8
9
10
Iteration 9418
1
2
3
4
5
6
7
8
9
10
Iteration 9419
1
Iteration 9420
1
2
3
4
5
6
7
8
9
10
Iteration 9421
1
Iteration 9422
1
Iteration 9423
1
2
3
4
5
6
7
8
9
10
Iteration 9424
1
2
3
4
5
6
7
8
9
10
I

1
Iteration 9691
1
2
3
4
5
6
7
8
9
10
Iteration 9692
1
2
3
4
5
6
7
8
9
10
Iteration 9693
1
2
3
4
5
6
7
8
9
10
Iteration 9694
1
Iteration 9695
1
2
3
4
5
6
7
8
9
10
Iteration 9696
1
Iteration 9697
1
Iteration 9698
1
2
3
4
5
6
7
8
9
10
Iteration 9699
1
Iteration 9700
1
2
3
4
5
6
7
8
9
10
Iteration: 9700; Percent complete: 97.0%; Average loss: 3.3709
Iteration 9701
1
Iteration 9702
1
Iteration 9703
1
2
3
4
5
6
7
8
9
10
Iteration 9704
1
Iteration 9705
1
Iteration 9706
1
Iteration 9707
1
2
3
4
5
6
7
8
9
10
Iteration 9708
1
Iteration 9709
1
2
3
4
5
6
7
8
9
10
Iteration 9710
1
2
3
4
5
6
7
8
9
10
Iteration 9711
1
Iteration 9712
1
2
3
4
5
6
7
8
9
10
Iteration 9713
1
Iteration 9714
1
2
3
4
5
6
7
8
9
10
Iteration 9715
1
2
3
4
5
6
7
8
9
10
Iteration 9716
1
Iteration 9717
1
Iteration 9718
1
Iteration 9719
1
2
3
4
5
6
7
8
9
10
Iteration 9720
1
2
3
4
5
6
7
8
9
10
Iteration 9721
1
2
3
4
5
6
7
8
9
10
Iteration 9722
1
2
3
4
5
6
7
8
9
10
Iteration 9723
1
Iteration 9724
1
2
3
4
5
6
7
8
9
10
Iteration 9725


Iteration 9997
1
2
3
4
5
6
7
8
9
10
Iteration 9998
1
Iteration 9999
1
2
3
4
5
6
7
8
9
10
Iteration 10000
1
2
3
4
5
6
7
8
9
10
Iteration: 10000; Percent complete: 100.0%; Average loss: 3.1782


In [36]:
# Set dropout layers to eval mode
forward_encoder.eval()
forward_decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(forward_encoder, forward_decoder)

# Begin chatting
evaluateInput(forward_encoder, forward_decoder, searcher, voc)

> hello
Bot: hello is this ? i d like to speak to mr doff .
> How are you
Bot: i m fine thanks . you you .
> why
Bot: i like to think the business is to
> do you like watching movies
Bot: well there s a movie on tv and i me to la .
> What is your name
Bot: my name is james .
> my name is james .
Bot: what s your nationality ? ? ?
> what s your nationality ? ? ?
Bot: american . .
> american . .
Bot: what s your relationship with the victim ?
> what s your relationship with the victim ?
Bot: i don t know him . him
> i don t know him . him
Bot: why did you attack the victim ?
> why did you attack the victim ?
Bot: i had t heard .
> i had t heard .
Bot: what you was her last last last ?
> what you was her last last last ?
Bot: i was just an appointment . was .
> i was just an appointment . was .
Bot: what did you want to do ?
> what did you want to do ?
Bot: i m not hungry . .
> i m not hungry . .
Bot: you re ? . some .
> you re ? . some .
Bot: i m glad you know what you mean .
> i m glad 

KeyboardInterrupt: Interrupted by user